In [2]:
import os
import cv2
import numpy as np
import multiprocessing
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess your dataset
def load_dataset():
    data_dir = "./dataset"
    categories = ["with_mask", "without_mask"]
    data = []
    labels = []

    for category in categories:
        path = os.path.join(data_dir, category)
        class_label = categories.index(category)

        for img_name in os.listdir(path):
            try:
                img_path = os.path.join(path, img_name)

                if not img_path.lower().endswith((".png", ".jpg", ".jpeg")):
                    continue

                img = cv2.imread(img_path)
                if img is None:
                    continue

                img = cv2.resize(img, (64, 64))
                data.append(img)
                labels.append(class_label)

            except Exception as e:
                print(f"Error loading image {img_name}: {e}")

    data = np.array(data) / 255.0
    labels = np.array(labels)
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
    y_train = to_categorical(y_train, 2)
    y_test = to_categorical(y_test, 2)
    return X_train, X_test, y_train, y_test


In [3]:
import matplotlib.pyplot as plt
from tqdm.keras import TqdmCallback
import time

def train_model_cpu(learning_rate):
    import matplotlib.pyplot as plt
    from tqdm.keras import TqdmCallback

    print(f"\nTraining with learning rate: {learning_rate}")
    start_time = time.time()

    X_train, X_test, y_train, y_test = load_dataset()

    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(2, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train,
                        validation_data=(X_test, y_test),
                        epochs=5,
                        batch_size=32,
                        verbose=1)

    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    end_time = time.time()
    duration = end_time - start_time

    print(f"Learning Rate: {learning_rate} | Accuracy: {acc:.4f} | Time: {duration:.2f} sec")

    # Save history & plot
    plt.figure(figsize=(10, 4))

    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Acc')
    plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.title(f'Accuracy (LR={learning_rate})')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f'Loss (LR={learning_rate})')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.savefig(f"performance_lr_{learning_rate}.png")
    plt.show()

    return {"lr": learning_rate, "accuracy": acc, "time": duration}


c:\Users\Satish\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

if __name__ == '__main__':
    learning_rates = [0.001, 0.0005]  # You can test with more if you want
    with multiprocessing.Pool(processes=2) as pool:
        pool.map(train_model_cpu, learning_rates)
